In [ ]:
# =========================================================
# ✅ YOLOv9m CCTV Fine-tuning Script
# ✅ Train from pretrained yolov9m.pt on custom dataset
# =========================================================

import os
import cv2
import torch
import random
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
from ultralytics import YOLO

# -----------------------------
# Check versions
# -----------------------------
print("✅ Albumentations version:", A.__version__)
print("✅ Torch version:", torch.__version__)

# =========================================================
# ✅ Set global seed for reproducibility
# =========================================================
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)

torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True  # faster training

# =========================================================
# ✅ Albumentations augmentations
# =========================================================
train_transform = A.Compose([
    A.RandomResizedCrop(size=(640, 640), scale=(0.65, 1.0), ratio=(0.9, 1.1), p=1.0),
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.04, scale_limit=0.12, rotate_limit=8,
                       border_mode=cv2.BORDER_CONSTANT, value=0, p=0.7),
    A.RandomBrightnessContrast(brightness_limit=0.25, contrast_limit=0.25, p=0.6),
    A.HueSaturationValue(hue_shift_limit=8, sat_shift_limit=12, val_shift_limit=12, p=0.5),
    A.CLAHE(clip_limit=2.0, tile_grid_size=(8, 8), p=0.35),
    A.RandomGamma(gamma_limit=(80, 120), p=0.4),
    A.MotionBlur(blur_limit=(3, 7), p=0.35),
    A.GaussNoise(var_limit=(5, 18), p=0.45),
    A.Defocus(radius=(1, 3), p=0.30),
    A.ImageCompression(quality_lower=30, quality_upper=70, p=0.40),
    ToTensorV2()
],
    bbox_params=A.BboxParams(
        format='yolo',
        label_fields=['class_labels'],
        min_visibility=0.10
    )
)

# =========================================================
# ✅ Dataset and save directory
# =========================================================
data_yaml = "/kaggle/input/vioelnceweapondetectiond/data.yaml"
save_dir = "/kaggle/working/yolov9m_cctv_aug"

# =========================================================
# ✅ Load pretrained YOLOv9m
# =========================================================
model = YOLO("yolov9m.pt")

# Set light augmentations in model
model.overrides['mosaic'] = 0.15
model.overrides['mixup'] = 0.10
model.overrides['copy_paste'] = 0.0

# =========================================================
# ✅ Apply Albumentations transforms during training
# =========================================================
def custom_dataloader_hook(dataset):
    dataset.transforms = train_transform
    return dataset

model.add_callback("on_fit_epoch_start",
                   lambda trainer: custom_dataloader_hook(trainer.train_loader.dataset))

# =========================================================
# ✅ Train from scratch on your dataset
# =========================================================
results = model.train(
    data=data_yaml,
    epochs=200,
    imgsz=640,
    batch=8,
    device=0,
    project=save_dir,
    name="yolov9m_cctv_high_accuracy",
    workers=2,
    lr0=0.002,
    optimizer="AdamW",
    pretrained=True,   # load yolov9m.pt
    val=True,
    exist_ok=True,
    patience=25
)

# =========================================================
# ✅ Validation
# =========================================================
metrics = model.val()
print("\n📊 Validation Metrics:")
print(f"Precision:   {metrics.box.pr:.4f}")
print(f"Recall:      {metrics.box.re:.4f}")
print(f"mAP50:       {metrics.box.map50:.4f}")
print(f"mAP50-95:    {metrics.box.map:.4f}")

print("\n✅ Training Completed")

# =========================================================
# ✅ Export to ONNX (optional)
# =========================================================
model.export(format="onnx", dynamic=True)


In [ ]:
# =========================================================
# ✅ YOLOv9m CCTV Resume Training Script
# ✅ (Continues from last.pt checkpoint)
# =========================================================

import os
import cv2
import torch
import random
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
from ultralytics import YOLO

print("✅ Albumentations", A.__version__)
print("✅ Torch", torch.__version__)

# =========================================================
# ✅ GLOBAL SEED (for same augmentations)
# =========================================================
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)

# Faster training mode (non-deterministic, but same aug)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True

# =========================================================
# ✅ CCTV / SMALL OBJECT SAFE AUGMENTATION (same as before)
# =========================================================
train_transform = A.Compose([
    A.RandomResizedCrop(size=(640, 640), scale=(0.65, 1.0), ratio=(0.9, 1.1), p=1.0),
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(
        shift_limit=0.04,
        scale_limit=0.12,
        rotate_limit=8,
        border_mode=cv2.BORDER_CONSTANT,
        value=0,
        p=0.7
    ),
    A.RandomBrightnessContrast(brightness_limit=0.25, contrast_limit=0.25, p=0.6),
    A.HueSaturationValue(hue_shift_limit=8, sat_shift_limit=12, val_shift_limit=12, p=0.5),
    A.CLAHE(clip_limit=2.0, tile_grid_size=(8, 8), p=0.35),
    A.RandomGamma(gamma_limit=(80, 120), p=0.4),
    A.MotionBlur(blur_limit=(3, 7), p=0.35),
    A.GaussNoise(var_limit=(5, 18), p=0.45),
    A.Defocus(radius=(1, 3), p=0.30),
    A.ImageCompression(quality_lower=30, quality_upper=70, p=0.40),
    ToTensorV2()
],
    bbox_params=A.BboxParams(
        format='yolo',
        label_fields=['class_labels'],
        min_visibility=0.10
    )
)

# =========================================================
# ✅ Paths
# =========================================================
data_yaml = "/kaggle/input/vioelnceweapondetectiond/data.yaml"
save_dir = "/kaggle/working/yolov9m_cctv_aug_runs"
resume_checkpoint = os.path.join(save_dir, "/kaggle/input/last-wights/last (11).pt")

# =========================================================
# ✅ Load model from last.pt (resume)
# =========================================================
print(f"🔄 Resuming training from checkpoint: {resume_checkpoint}")
model = YOLO(resume_checkpoint)  # automatically loads optimizer, epoch, etc.

# Light mix augmentation (same as before)
model.overrides['mosaic'] = 0.15
model.overrides['mixup'] = 0.10
model.overrides['copy_paste'] = 0.0

# =========================================================
# ✅ Keep same augmentation hook
# =========================================================
def custom_dataloader_hook(dataset):
    dataset.transforms = train_transform
    return dataset

model.add_callback("on_fit_epoch_start",
                   lambda trainer: custom_dataloader_hook(trainer.train_loader.dataset))

# =========================================================
# ✅ RESUME TRAINING
# =========================================================
results = model.train(
    data=data_yaml,
    epochs=200,                 # total target epochs (YOLO auto-continues from last one)
    imgsz=640,
    batch=8,
    device=0,
    project=save_dir,
    name="yolov9m_cctv_high_accuracy",
    workers=2,
    lr0=0.002,
    optimizer="AdamW",
    pretrained=True,
    val=True,
    exist_ok=True,
    patience=25,
    resume=True                 # ✅ key flag to resume from last.pt
)

# =========================================================
# ✅ VALIDATION (after resume)
# =========================================================
metrics = model.val()
print("\n📊 Validation Metrics:")
print(f"Precision:   {metrics.box.pr:.4f}")
print(f"Recall:      {metrics.box.re:.4f}")
print(f"mAP50:       {metrics.box.map50:.4f}")
print(f"mAP50-95:    {metrics.box.map:.4f}")

print("\n✅ Training Resumed and Completed Successfully")

# =========================================================
# ✅ Export to ONNX (optional)
# =========================================================
model.export(format="onnx", dynamic=True)
